# Common Numerical Integration Algorithms
Computing a numerical integral (or quadrature) is a typical problem that appears everywhere in numerical computing. The simple task of computing the error function, $f(x) = \int_{0}^x e^{-t^2}$, requires the application of numerical techniques to calculate this (improper integral) over an arbitrary range.  

In any numerical computing course you will start by exploring the trapezoidal and the simpsons rule. Here we are jus going to give the general formulation and then go directly to the functions given by scipy

Let's compare several of the typical algorithms: https://www.geogebra.org/m/WsZgaJcc, and a beautiful visualization at https://www.youtube.com/watch?v=FnJqaIESC2s

## Trapezoidal rule
Here the function between two points is approximated by a straight line and the te area covered by the trapezoid is added up. See: https://en.wikipedia.org/wiki/Trapezoidal_rule

|<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Integration_num_trapezes_notation.svg/330px-Integration_num_trapezes_notation.svg.png" /> |<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Trapezium2.gif/330px-Trapezium2.gif" />|
|-|-|
|1-order interpolation|Better results for more intervals|


The integral between two poins, $a$ and $b$, after dividing its lenght in $N$ intervals (N+1 points, regular partition) of size $\Delta x = \frac{b-a}{N}$, is given by
$$\int_a^b f(x) dx \simeq \Delta x \left( \frac{f(x_0)}{2} + \sum_{k=1}^{N-1} f(x_k) +\frac{f(x_N)}{2}\right) + O(N^{-1}),$$
where $x_k = x_0 + k \Delta x$, $x_0 = a$, and $x_N=b$.

Now, please create a function to compute the integral of a function using the trapezoid algorithm. Write the declaration and implementation on in `integration.h` and `integration.cpp`, respectively. It must fulfill the following tests
```c++
std::cout << trapezoid(0.0, 1.0, 10, fun) << "\n";
std::cout << trapezoid(0.0, 1.0, 1000, fun) << "\n";
std::cout << trapezoid(0.0, 1.0, 10000000, fun) << "\n";
```
with results
```bash
0.3350000000000001
0.33333349999999995
0.33333333333333487
```
where fun is the square function.



## **Exercise**
Now use your implementation to compute the error function as a function of $x$, using the partition shown. Your program must print also the relative error, comparing with the "exact" value from https://en.cppreference.com/w/cpp/numeric/math/erf . Print your data to `erf_data.txt` (print the x value, the numerical value of the integral, and the relative error)

\begin{equation}
f(x) = \frac{2}{\sqrt{\pi}}\int_0^x e^{-t^2} dt
\end{equation}

$x \in [0, 10]$, $\Delta x = 0.1$

$f(x=0), f(x=0+0.1), f(x=0+2\times 0.1), f(x=0+3\times 0.1), \ldots$

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')

x, erf, error = np.getfromtxt('erf_data.txt', unpack=True)
fig, ax = plt.subplots(1, 2)
ax[0].plot(x, erf, '-o', label="err function")
ax[0].legend()
ax[1].plot(x, error, '-s', label="relative error")
ax[1].legend()
ax[1].loglog()

## Simpsons rule
In this case, the method uses three discrete points and approximate the function as a parabola (second order polynomial), to improve the numerical integral computation. See: https://en.wikipedia.org/wiki/Simpson%27s_rule

|<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Simpsons_method_illustration.svg/440px-Simpsons_method_illustration.svg.png" width=300 /> |<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Simpsonsrule2.gif/440px-Simpsonsrule2.gif" />|
|-|-|
|2-order interpolation|Better results for more intervals|

Now you compute the integral as
\begin{equation}
\int_a^b f(x) dx \simeq \frac{\Delta x}{3}\left( f(x_0) + 4 \sum_{k=1}^{N/2} f(x_{2k-1}) + 2\sum_{k=1}^{N/2-1} f(x_{2k}) + f(x_N)\right) + O(N^{-3}).
\end{equation}
Notice that $N$ must be even.

Now let's implement it. Please add a new function to the `integration.*` files , that implements the simpsons rule.



In [ ]:
std::cout << simpson(0.0, 1.0, 10) << "\n";

Should give

In [ ]:
0.3333333333333333

# Romberg method
The [Romberg Method](https://en.wikipedia.org/wiki/Romberg%27s_method) applies Richardson extrapolation to the given integration technique. The implementation is completely analogous as for the derivatives case.

Implement it for the previous method shown and compare the errors as a function of $N$, the number of intervals. Use the err function and fix $x = 0.54432$.





## Other techniques
There are ways to improve the previous algorithms to decrease the error: using  higher order decompositions, using [Adaptative techniques](https://en.wikipedia.org/wiki/Adaptive_Simpson%27s_method), using [Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature), mixing some of these techniques, etc. But implementing those methods by hand would be a daunting task.

NOTE: When you have tabular data, you will be almost forced to use either the trapezoid or the simpsons method. You could also first interpolate the data and then integrate.

# Gaussian quadrature: Introduction to vectors
[Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature?useskin=vector) is a very clever technique that allows to compute integrals with a high precision and minimal number of points. First, notice that any numerical integral method can be written as
$$
I \simeq \sum w_i f(x_i),
$$
where $x_i$ are the evaluation points and $w_i$ the associated weigths. What if you don't fix the $x_i$ to a uniform spacing, but, instead, try to compute both their optimal positions plus the optimal weights so you compute exactly the integral for an interpolating polynomial of degree $2n-1$, using $n$ points? that is the basic idea behind the method. The modern formulation, thanks to Jacobi, uses orthogonal polynomials, usually in the [-1, 1] domain (so you need to re-scale your data).

<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/660px-Comparison_Gaussquad_trapezoidal.svg.png" alt="Image Description" width="600">
    <figcaption>From: "https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/660px-Comparison_Gaussquad_trapezoidal.svg.png"</figcaption>
</div>
In this image, it is illustrated that although the Gaussian method uses only two points, it is exact since the integration is done on a 3-degree polynomial.


For the Gauss-Legendre formulation, you can check the weights at https://en.wikipedia.org/wiki/Gaussian_quadrature?useskin=vector#Gauss%E2%80%93Legendre_quadrature

To succesfully apply the Gaussian quadrature technique to a general integral on the interval [a, b], we need to map it to the interval [-1, 1], so we can use the mapping
$$
x = \frac{b-a}{2}x' + \frac{b+a}{2},
$$
therefore
$$
\int_a^b f(x) dx \simeq \sum w_i f(\frac{b-a}{2}x' + \frac{b+a}{2})
$$
using the $w_i$ from the method.

## Example using a vector to store the weights.
For an increasing number of points, you will need to use many variables to store the weights. For a degree $n=5$, you will need five variables to store the weights, five more for the points, and then manually process that. It will be much better to store them in an data structure that allow us to handle the operations in a generic way. In this case, it will be a `std::vector`, which allows us to store data, contiguously in memory, and process it using simple for loops (SIMD).

Our declaration could be
```c++
double gauss_5(double a, double b, fptr fun);
```
and its implementation (you need to finish it)
```c++
double gauss_5(double a, double b, fptr fun)
{
    std::vector<double> x(5); // store five points in the array
    std::vector<double> w(5); // store five weights in the array
    // initialize the data, sorted from left to right
    x[0] = -std::sqrt(5 + 2std::sqrt(10/7));
    x[1] = -std::sqrt(5 - 2std::sqrt(10/7));
    x[2] = 0.0;
    // ...
    w[0] = (322 - 13*std::sqrt(70))/(900);
    w[1] = (322 + 13*std::sqrt(70))/(900);
    w[2] = 128.0/255.0;
    //...
    // compute the integral
    double result = 0.0;
    for (int ii = 0; ii < 5; ++ii) {
        result += w[ii]*fun((b-a)*x[ii]/2 + (b+a)/2);
    }
    return result;
}
```

Notice how easy is to process data in an array.

Now finish the implementation, and print the error, as a function of $x$, computing the err function, comparing the relative errors  with trapezoid, simpson, trapezoid+richardson, simpson+trapezoid, gauss_5. Plot it.